## API Featching for CQC rating in Dudley Care Homes

In [53]:
import requests
import json

In [ ]:
# Base URL for the CQC API
base_url = "https://api.service.cqc.org.uk/public/v1/locations"

# Parameters for the API request
params = {
    "localAuthority": "Dudley",  
    "careHome": "Y",            
    "perPage": 100               
}

# Subscription key for authentication
subscription_key = "6c0491618d764af0a892e9a51c9666ce"

# Headers for the API request
headers = {
    # User-Agent header to avoid getting blocked
    "User-Agent": "Mozilla/5.0",  
    # Subscription key for authentication
    "Ocp-Apim-Subscription-Key": subscription_key  
}

def fetch_data(url, params, headers):
    """
    Fetch data from the API.
    """
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        # Return the JSON response if successful
        return response.json()  
    return None

def fetch_care_home_ratings(location_id, headers):
    """
    Fetch overall ratings for a care home by its location ID.
    """
    url = f"https://api.service.cqc.org.uk/public/v1/locations/{location_id}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        rating = data.get('currentRatings', {}).get('overall', {}).get('rating')
        return rating
    return None

# Fetch data from the CQC API
data = fetch_data(base_url, params, headers)

# Check if data was successfully fetched
if data:
    # Process and print the data as needed
    for location in data.get('locations', []):
        name = location.get('locationName')
        location_id = location.get('locationId')
        rating = fetch_care_home_ratings(location_id, headers)
        if rating is not None:
            print(f"Care Home: {name}, Rating: {rating}")


Care Home: Cygnet House, Rating: Good
Care Home: 60 Olive Lane, Rating: Good
Care Home: Arcare for Forte Limited, Rating: Good
Care Home: Hollybush House Nursing Home, Rating: Good
Care Home: Woodview House Nursing Home, Rating: Requires improvement
Care Home: Commonside Care Limited - 73 Commonside, Rating: Good
Care Home: Stanford House, Rating: Requires improvement
Care Home: Tudor Manor, Rating: Inadequate
Care Home: Attwood Street, Rating: Good
Care Home: Langstone Road, Rating: No published rating
Care Home: Stickley Lane, Rating: Requires improvement
Care Home: Edmore House Residential Home, Rating: Good
Care Home: Belvidere Residential Care Home, Rating: Requires improvement
Care Home: Woodlands Gate Rest Home, Rating: Good
Care Home: Two Gates House, Rating: Good
Care Home: Eve House, Rating: No published rating
Care Home: Nelson House, Rating: Requires improvement
Care Home: Mr and Mrs O'Donnell, Rating: Inadequate
Care Home: Halas Homes, Rating: Good
Care Home: Castlewellan 

In [56]:
# Iterate over the locations array and extract information
for location in data.get('locations', []):
    name = location.get('locationName')
    postal_code = location.get('postalCode')
    print(f"Care Home: {name}, Postal Code: {postal_code}")

Care Home: Cygnet House, Postal Code: DY8 3UU
Care Home: Parkfield Grange, Postal Code: DY8 1QH
Care Home: 60 Olive Lane, Postal Code: B62 8LZ
Care Home: Arcare for Forte Limited, Postal Code: WV14 9QB
Care Home: Hollybush House Nursing Home, Postal Code: DY8 4HZ
Care Home: Woodview House Nursing Home, Postal Code: B63 1EN
Care Home: Commonside Care Limited - 73 Commonside, Postal Code: DY5 4AJ
Care Home: Stanford House, Postal Code: DY3 1ST
Care Home: Tudor Manor, Postal Code: DY8 3XF
Care Home: Attwood Street, Postal Code: B63 3UE
Care Home: Langstone Road, Postal Code: DY1 2NJ
Care Home: Stickley Lane, Postal Code: DY3 2JQ
Care Home: Edmore House Residential Home, Postal Code: DY2 7TB
Care Home: Belvidere Residential Care Home, Postal Code: DY1 2DH
Care Home: Woodlands Gate Rest Home, Postal Code: DY9 0RS
Care Home: Two Gates House, Postal Code: B63 2LJ
Care Home: Beatrice House, Postal Code: DY5 4HG
Care Home: Eve House, Postal Code: DY5 4HW
Care Home: Nelson House, Postal Code: DY